In [12]:
import numpy as np
import pandas as pd
import logging
import os
from datetime import timedelta
from pathlib import Path

from tinkoff.invest import CandleInterval, Client
from tinkoff.invest.utils import now
from tinkoff.invest.caching.market_data_cache.cache import MarketDataCache
from tinkoff.invest.caching.market_data_cache.cache_settings import (
    MarketDataCacheSettings,
)

# Рабочая версия tinkoff.invest '0.2.0b59'

Вводим токен, получаем список доступных аккаунтов

In [3]:
TOKEN0 = 't.UFRJ8SC9hafVOhFxEUY7yf1wZ1gGhwJp-WCp9o4rnEChHWns0c3jQ21eQwoOW_RurFqeZpss2scJkmMQnomJ9g'
TOKEN1 = 't.6nHltT1dYSfrVTIV9zF72fxDlB2sXJbRD6iJNpZXTFAN61rmD7m71xPp9ko12ta1JxA06em4YdN36xicnBmjWg'

token = TOKEN0

res = []
with Client(token) as client:
    accounts = client.users.get_accounts()

### Portfolio

Получаем портфель, список всех позиций на аккаунте

In [4]:
acc_id = accounts.accounts[0].id
acc_id
with Client(token) as client:
    port = client.operations.get_portfolio(account_id=acc_id) 

Получаем отдельные позиции, идентфикатор figi, их количество и цену

In [5]:
def get_id_base(TOKEN):
    with Client(TOKEN) as cl:
        instruments = cl.instruments
        market_data = cl.market_data
    
        l = []
        for method in ['shares', 'currencies', 'futures', 'bonds', 'etfs']:
            for item in getattr(instruments, method)().instruments:
                l.append({
                    'ticker': item.ticker,
                    'figi': item.figi,
                    'type': method,
                    'name': item.name,
                    'cur' : item.currency,
                    'lot' : item.lot
                })
    
        df = pd.DataFrame(l)
    return df

def ticker_to_figi(ticker, df):
    dfx = df[df['ticker'] == ticker]   
    if dfx.shape[0] > 0 :
        figi = dfx['figi'].iloc[0]
        return figi
    else:
        return None

def figi_to_ticker(figi, df):
    dfx = df[df['figi'] == figi]   
 
    if dfx.shape[0] > 0 :
        ticker = dfx['ticker'].iloc[0]      
        return ticker
    else :
        return None

def figi_to_name(figi, df):
    dfx = df[df['figi'] == figi]   
 
    if dfx.shape[0] > 0 :
        res = dfx['name'].iloc[0]      
        return res
    else :
        return None
    
    
def money_value(price):
    return price.units + price.nano / 1e9

#### База идентификаторов тиньков

In [6]:
base = get_id_base(token)

In [7]:
res = []
for pos in port.positions:
    ticker = figi_to_ticker(pos.figi, base)
    name = figi_to_name(pos.figi, base)
#    print("Figi: ", pos.figi, ticker, name, "Количество: ", pos.quantity.units, "Цена: ", pos.current_price.units + pos.current_price.nano / 1e9)
    
    res.append({
        'figi': pos.figi,
        'ticker': ticker,
        'name': name,
        'quantity' : pos.quantity.units,
        'price' : money_value(pos.current_price)
    })

df_port = pd.DataFrame(res)    
df_port = df_port.sort_values("ticker")    
df_port

,figi,ticker,name,quantity,price
3,BBG007N0Z367,AGRO,РусАгро,16,1393.400
8,BBG004S686N0,BANEP,Башнефть - привилегированные акции,29,2138.000
12,BBG00475K6C3,CHMF,Северсталь,39,1704.200
0,BBG004731032,LKOH,ЛУКОЙЛ,11,7513.000
5,BBG004S68507,MAGN,Магнитогорский металлургический комбинат,740,54.895
1,BBG004S681B4,NLMK,НЛМК,390,207.280
13,BBG004731354,ROSN,Роснефть,37,589.450
2,TCS00A107RZ0,RU000A107RZ0,ГК Самолет выпуск 13,40,999.900
11,BBG004730N88,SBER,Сбер Банк,220,300.400
7,BBG004S684M6,SIBN,Газпром нефть,47,829.800


### Figi to tiker

#### Получение исторических данных

In [9]:
def get_candles(figi, interval):
    res = []
    with Client(token) as client:
        settings = MarketDataCacheSettings(base_cache_dir=Path("market_data_cache_01"))
        market_data_cache = MarketDataCache(settings=settings, services=client)
        for candle in market_data_cache.get_all_candles(
            figi = figi,
            from_=now() - timedelta(days=180),
            interval= interval,
        ):
            price_row = [candle.open,  candle.high, candle.low, candle.close]
            price_row = [money_value(x) for x in price_row]
            res.append([candle.time] + price_row)    
        return res       
    
def get_open_price(candles):
    res = []
    for row in candles:
        sdate = row[0]
        sdate = sdate.strftime("%Y-%m-%d")
        res.append([sdate] + row[1:2]) 
    df = pd.DataFrame(res, columns = ['date', 'ticker'])
    df = df.set_index('date')
    
    return df

### Load account data

In [18]:
res = []

for pos in port.positions:
    print(pos.figi)        
    candles = get_candles(pos.figi, CandleInterval.CANDLE_INTERVAL_DAY)
    df =  get_open_price(candles)
    ticker = figi_to_ticker(pos.figi, base)
    
    if ticker == None:
        ticker = pos.figi
    df.columns = [ticker]
    res.append(df)
    
dfp = pd.concat(res, axis = 1)

BBG004731032
BBG004S681B4
TCS00A107RZ0
BBG007N0Z367
BBG00475KHX6
BBG004S68507
BBG004RVFFC0
BBG004S684M6
BBG004S686N0
BBG004S681M2
RUB000UTSTOM
BBG004730N88
BBG00475K6C3
BBG004731354


In [10]:
dfp = dfp.dropna(axis = 1)

### Portfolio

In [17]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

In [18]:

def weights_to_df(cleaned_weights):
    dfw = pd.DataFrame.from_dict([cleaned_weights]).transpose()
    dfw.columns = ['W']
    dfw = dfw[dfw['W'] > 0]
    return dfw

def final_sums(df, total):
    xx = round(df * total, -1)
    xx = xx[xx['W'] > 0]
    xx = xx.sort_values("W")
    return xx

In [13]:
avg_returns = expected_returns.mean_historical_return(dfp)
cov_mat = risk_models.sample_cov(dfp)
cov_mat

,BSPB,RASP,FEES,VTBR,NMTP,GAZP,BELU,MSNG,PHOR,SIBN,...,MRKS,NKNCP,MTLRP,POSI,SBER,ROLO,MTLR,CBOM,CHMF,ROSN
BSPB,0.320103,0.025596,0.036976,0.063678,0.180115,0.016250,0.059937,0.028329,0.005670,0.013616,...,0.042697,0.038346,0.036242,0.021435,0.030166,0.128216,0.022772,0.027879,0.048142,0.035551
RASP,0.025596,0.126472,0.063543,0.034831,0.053821,0.017360,0.021382,0.033503,0.020385,0.027404,...,0.029235,0.023464,0.058119,0.023455,0.035512,0.026812,0.072760,0.029343,0.026007,0.036616
FEES,0.036976,0.063543,0.166530,0.058056,0.086871,0.029865,0.030009,0.056246,0.031041,0.036314,...,0.178779,0.046363,0.047157,0.037806,0.044278,0.153163,0.050305,0.033450,0.045790,0.044192
VTBR,0.063678,0.034831,0.058056,0.096868,0.061370,0.027556,0.022304,0.030639,0.019970,0.028979,...,0.077647,0.034537,0.043716,0.034313,0.039720,0.069346,0.054236,0.030105,0.049601,0.036441
NMTP,0.180115,0.053821,0.086871,0.061370,0.501493,0.033858,0.087459,0.068519,0.031719,0.028442,...,0.153015,0.054862,0.070927,0.035677,0.045106,0.163982,0.069643,0.039238,0.052439,0.062204
GAZP,0.016250,0.017360,0.029865,0.027556,0.033858,0.023750,0.011319,0.019034,0.008838,0.016017,...,0.035770,0.018065,0.024742,0.013531,0.016288,0.042855,0.024246,0.011621,0.015067,0.017245
BELU,0.059937,0.021382,0.030009,0.022304,0.087459,0.011319,0.080215,0.036961,0.010593,0.013021,...,0.041696,0.029500,0.048876,0.016835,0.018593,0.048994,0.031431,0.019834,0.021180,0.024544
MSNG,0.028329,0.033503,0.056246,0.030639,0.068519,0.019034,0.036961,0.105078,0.014519,0.024935,...,0.083868,0.056586,0.041330,0.025881,0.026763,0.172510,0.028101,0.026360,0.025836,0.031135
PHOR,0.005670,0.020385,0.031041,0.019970,0.031719,0.008838,0.010593,0.014519,0.033807,0.018167,...,0.030524,0.014317,0.024840,0.011083,0.016354,0.019908,0.025875,0.007513,0.015568,0.019189
SIBN,0.013616,0.027404,0.036314,0.028979,0.028442,0.016017,0.013021,0.024935,0.018167,0.074831,...,0.053799,0.028970,0.030986,0.018533,0.023808,0.043941,0.023747,0.017130,0.015155,0.028690


In [112]:
ef = EfficientFrontier(avg_returns, cov_mat)
weights = ef.max_sharpe()
ef.portfolio_performance(verbose=True)
cleaned_weights = ef.clean_weights()
dfw = weights_to_df(cleaned_weights)
final_sums(dfw, 10000)

Expected annual return: 129.1%
Annual volatility: 19.3%
Sharpe Ratio: 6.59


,W
FLOT,650.0
TATN,1470.0
SIBN,4400.0
TBRU,1540.0
MRKS,10.0
ROLO,130.0
MTLR,1790.0


In [64]:
# get weights maximizing the Sharpe ratio
ef = EfficientFrontier(avg_returns, cov_mat)
weights = ef.min_volatility()
ef.portfolio_performance(verbose=True)
cleaned_weights = ef.clean_weights()
dfw = weights_to_df(cleaned_weights)
final_sums(dfw, 10000)

Expected annual return: 6.2%
Annual volatility: 4.3%
Sharpe Ratio: 0.97


,W
CNYRUB_TOM,970.0
GMKN,160.0
GAZP,20.0
PHOR,270.0
TBRU,8440.0
MTSS,100.0
ROLO,40.0


### Model

In [56]:
risk_methods = [
    "sample_cov",
    "semicovariance",
    "exp_cov",
    "ledoit_wolf",
    "ledoit_wolf_constant_variance",
    "ledoit_wolf_single_factor",
    "ledoit_wolf_constant_correlation",
    "oracle_approximating",
]

return_methods = [
    "mean_historical_return",
    "ema_historical_return",
    "capm_return", 
    ]

In [22]:

def calc_frontier(df_period, risk_method, ret_method = "mean_historical_return", span = 180):

    if ret_method == "ema_historical_return":
        mu = expected_returns.return_model(df_period, method=ret_method, span = span)
    else:
        mu = expected_returns.return_model(df_period, method=ret_method)
  
    cov_mat = risk_models.risk_matrix(df_period, method=risk_method)        
    ef = EfficientFrontier(mu, cov_mat)
    return ef


def calc_weights(ef, opt_type, par, verbose = False):

    if opt_type == 'max_sharpe':
        try:
            weights = ef.max_sharpe()
        except:
            if verbose:
                print("Non-convex optimize!")
            weights = ef.nonconvex_objective(
            objective_functions.sharpe_ratio,
            objective_args=(ef.expected_returns, ef.cov_matrix),
            weights_sum_to_one=True,
        )  
    elif opt_type == 'efficient_risk':
        weights = ef.efficient_risk(par) 

    ef.portfolio_performance(verbose=verbose)
    cleaned_weights = ef.clean_weights(cutoff=0.0001)
    dfw = weights_to_df(cleaned_weights)
    dfw['W'] = dfw['W']/dfw['W'].sum()
    return dfw


In [192]:
drops = []

columns = [x for x in dfp.columns if x not in drops]
dfpx = dfp[columns]

In [21]:
#ef = calc_frontier(dfp, "semicovariance",  "ema_historical_return", span = 90)
ef = calc_frontier(dfpx, "ledoit_wolf",  "ema_historical_return", span = 90)
#ef = calc_frontier(dfp, "ledoit_wolf_constant_correlation",  "mean_historical_return")
#ef = calc_frontier(dfp, "exp_cov",  "capm_return")
 
#ef = calc_frontier(dfp, "ledoit_wolf_constant_variance",  "mean_historical_return")
#ef = calc_frontier(dfp, "sample_cov")
#weights = ef.max_sharpe()


dfw = calc_weights(ef, 'max_sharpe', 0, verbose = True)      
#dfw = calc_weights(ef, 'efficient_risk', 0.3, verbose = True) 

NameError: name 'dfpx' is not defined

In [198]:
dfx = final_sums(dfw, 10000)
dfx = dfx.sort_values("W", ascending = False)
dfx

,W
CHMF,3400.0
MTLR,2220.0
FLOT,2020.0
TBRU,1310.0
SIBN,580.0
TATN,460.0


## All shares

In [10]:
dfx = base[base["type"] == "shares"]
dfx = dfx[dfx["cur"] == "rub"]

In [13]:
res = []
for row in dfx.iterrows():
    pos = row[1]
#    print(pos.figi)        
    candles = get_candles(pos.figi, CandleInterval.CANDLE_INTERVAL_DAY)
    df =  get_open_price(candles)
    ticker = figi_to_ticker(pos.figi, base)
    
    if ticker == None:
        ticker = pos.figi
    df.columns = [ticker]
    res.append(df)
    
dfp = pd.concat(res, axis = 1)


In [14]:
dfp = dfp.dropna(axis = 1)
dfp

,DSKY,ENPG,NKNCP,OBNEP,IRKT,MRKV,VSMO,UNAC,LSRG,TTLK,...,ABRD,CNTLP,PIKK,ROSN,MTLRP,TRMK,MRKU,MRKY,KRKNP,CHMF
date,,,,,,,,,,,,,,,,,,,,,
2023-09-13,67.04,529.0,93.46,1300.00,135.80,0.08115,52540.0,2.4210,729.8,1.6220,...,337.2,16.06,794.0,563.40,227.90,244.98,0.5120,0.11250,14520.0,1323.0
2023-09-14,66.94,516.8,90.72,1000.00,126.00,0.07610,53480.0,2.2700,722.4,1.4950,...,330.2,15.62,782.0,555.15,222.30,240.68,0.4960,0.10850,14980.0,1300.0
2023-09-15,66.50,506.0,87.76,1100.00,111.80,0.06910,51260.0,2.0290,707.0,1.2500,...,328.2,13.94,777.0,556.55,220.70,234.00,0.4682,0.09460,14100.0,1295.0
2023-09-18,66.32,521.2,89.28,1200.00,127.90,0.07460,52020.0,2.4315,725.2,1.4660,...,334.6,14.88,782.8,565.00,227.00,245.00,0.4950,0.10380,14300.0,1313.0
2023-09-19,65.52,519.0,88.50,1060.00,118.75,0.07320,50900.0,2.1200,722.0,1.3800,...,320.0,14.52,767.5,558.60,223.50,244.56,0.4666,0.10245,14200.0,1292.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-01,51.46,438.0,78.30,949.34,77.00,0.06495,38260.0,1.4300,980.0,1.0625,...,265.4,11.84,875.0,578.05,317.60,220.50,0.4134,0.10065,12300.0,1652.0
2024-03-04,51.00,440.8,78.98,1012.92,75.00,0.06425,37900.0,1.4300,985.0,1.0745,...,261.0,11.88,868.7,578.90,324.80,217.50,0.4130,0.09950,12360.0,1674.6
2024-03-05,50.06,452.0,78.00,981.90,73.80,0.06495,37800.0,1.4075,995.0,1.1090,...,256.2,11.64,858.0,582.60,323.55,217.50,0.4170,0.10330,13540.0,1674.2


In [15]:
drops = ["GTRK", "SFIN", "ORUP", "UWGN", "RKKE", "KROT", "APTK", "VEON-RX"]
drops = ["SFIN", "GTRK", "NTZL", "LSRG"]
columns = [x for x in dfp.columns if x not in drops]
dfp = dfp[columns]

In [23]:
#ef = calc_frontier(dfp, "semicovariance",  "ema_historical_return", span = 90)
#ef = calc_frontier(dfp, "ledoit_wolf",  "ema_historical_return", span = 180)
ef = calc_frontier(dfp, "ledoit_wolf",  "ema_historical_return", span = 90)
#ef = calc_frontier(dfp, "ledoit_wolf_constant_correlation",   "ema_historical_return", span = 90)

 
#ef = calc_frontier(dfp, "ledoit_wolf_constant_variance",  "ema_historical_return", span = 120)
#ef = calc_frontier(dfp, "sample_cov")
#weights = ef.max_sharpe()


dfw = calc_weights(ef, 'max_sharpe', 0, verbose = True)      
#dfw = calc_weights(ef, 'efficient_risk', 0.25, verbose = True) 

Expected annual return: 243.9%
Annual volatility: 21.6%
Sharpe Ratio: 11.22


In [24]:
dfw.sort_values("W", ascending = False)

,W
YNDX,0.209678
PIKK,0.178238
BANE,0.165588
WUSH,0.109269
MDMG,0.097079
RTKM,0.062409
CHMF,0.056159
GRNT,0.035660
RTKMP,0.028110
MGNT,0.017070


In [29]:
dfx = final_sums(dfw, 111000)
dfx['lot'] = 1
inds = dfx.index.values.tolist()

inds
x = base[base['ticker'].isin (inds)]
s = x[['ticker', 'lot']].set_index('ticker')
dfx['lot'] = s


In [36]:
prices = dfp.iloc[-1].T.loc[dfx.index]
dfx["price"] = prices
dfx["buy"] = np.round(dfx.W / (dfx.price * dfx.lot))
dfx["sum"]= dfx.price * dfx.buy * dfx.lot
dfx = dfx.sort_values("W", ascending = False)
dfx.to_csv("t.csv")

In [32]:
dfx.sum()

W        110990.0000
lot       10031.0000
price     19414.2276
buy       26958.0000
sum      109564.1704
dtype: float64